# WSL2 USB Camera Cheatsheet (PowerShell & Ubuntu)

Schneller Leitfaden, um USB-Kameras mit **WSL2** zu nutzen: Setup in **PowerShell**, Tests in **Ubuntu** mit `ffplay` und **OpenCV**.

**Hinweis:** Ersetze `X-Y` unten durch deine *BUSID* (z. B. `3-1`) aus `usbipd list`. 

## PowerShell (als Administrator)

```powershell
# Install usbipd-win (einmalig)
winget install usbipd

# USB-Geräte auflisten
usbipd list

# Gerät für WSL freigeben (bind)
usbipd bind --busid X-Y

# An WSL anhängen
usbipd attach --wsl --busid X-Y

# Persistentes Anhängen (Auto-Attach)
usbipd attach --wsl --busid X-Y --persist

# Von WSL trennen (bleibt geteilt)
usbipd detach --busid X-Y

# Freigabe aufheben (zurück an Windows)
usbipd unbind --busid X-Y

# Status prüfen
usbipd wsl list

# Quick-Reset, wenn der Stream hängt
usbipd detach --busid X-Y
usbipd unbind --busid X-Y
usbipd bind --busid X-Y
usbipd attach --wsl --busid X-Y --persist
```

> **Tipp (AV-Suiten):** Falls Streams abbrechen oder schwarz bleiben, Netzwerk/HTTPS-Überwachung testweise pausieren oder Ausnahmen setzen für:
> - `C:\Program Files\usbipd-win\usbipd.exe`
> - `C:\Windows\System32\wsl.exe`

## Ubuntu (WSL2) – Setup & Diagnose

```bash
sudo apt update
sudo apt install -y v4l-utils ffmpeg python3-opencv

# Devices & Formate ansehen
ls -l /dev/video*
v4l2-ctl --list-devices
v4l2-ctl -d /dev/video0 --list-formats-ext
```

### Robustes MJPEG-Profil setzen (Beispiel 640×360 @ 30 fps)
```bash
v4l2-ctl -d /dev/video0 --set-fmt-video=width=640,height=360,pixelformat=MJPG --set-parm=30
```

### Stream-Test mit ffplay (niedrige Latenz)
```bash
ffplay -hide_banner -loglevel warning -fflags nobuffer \
  -f v4l2 -input_format mjpeg -video_size 640x360 -framerate 30 -i /dev/video0
```

### Alternative: YUYV (unkomprimiert)
```bash
v4l2-ctl -d /dev/video0 --set-fmt-video=width=640,height=480,pixelformat=YUYV --set-parm=30
ffplay -hide_banner -loglevel warning -fflags nobuffer \
  -f v4l2 -input_format yuyv422 -video_size 640x480 -framerate 30 -i /dev/video0
```

### Optionales, leichtgewichtiges GUI-Tool (stabiler als *cheese*)
```bash
sudo apt install -y guvcview
guvcview -d /dev/video0
```

## Inline OpenCV Preview (direkt im Terminal einfügen)

```bash
python3 - <<'PY'
import cv2,time; DEV="/dev/video0"; cap=cv2.VideoCapture(DEV,cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC,cv2.VideoWriter_fourcc(*'MJPG'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640); cap.set(cv2.CAP_PROP_FRAME_HEIGHT,360)
cap.set(cv2.CAP_PROP_FPS,30)
if not cap.isOpened(): exit(print("Camera not found:",DEV))
print("Camera opened – [q] quits")
while True:
    ok,f=cap.read()
    if not ok: break
    cv2.imshow("OpenCV Preview",f)
    if cv2.waitKey(1)&0xFF==ord('q'): break
cap.release(); cv2.destroyAllWindows()
PY
```

## Inline OpenCV Snapshot + FPS Quick Check

```bash
python3 - <<'PY'
import cv2,time,hashlib
DEV="/dev/video0"; W,H,FPS=640,360,30; FOURCC='MJPG'
cap=cv2.VideoCapture(DEV, cv2.CAP_V4L2)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*FOURCC))
cap.set(cv2.CAP_PROP_FRAME_WIDTH,W); cap.set(cv2.CAP_PROP_FRAME_HEIGHT,H); cap.set(cv2.CAP_PROP_FPS,FPS)
if not cap.isOpened(): exit(print("Camera not found:",DEV))
for _ in range(5): cap.read(); time.sleep(0.02)
n=t0=0.0; import time as _t; t0=_t.time(); n=0
while n<30:
    ok,_=cap.read()
    if not ok: break
    n+=1
est=(n/(_t.time()-t0)) if n else 0.0
ok,frame=cap.read()
if not ok: exit(print("No frame received"))
path="/tmp/opencv_probe.jpg"; cv2.imwrite(path,frame,[int(cv2.IMWRITE_JPEG_QUALITY),90])
md5=hashlib.md5(open(path,"rb").read()).hexdigest()
print(f"Camera OK | {frame.shape[1]}x{frame.shape[0]}  ~{est:.1f} fps  -> {path}  md5={md5}")
cap.release()
PY
```

## Troubleshooting (Ubuntu)

```bash
# Rechte setzen
sudo usermod -aG video $USER
newgrp video
sudo chmod a+rw /dev/video0

# Wenn Windows-Apps die Kamera blockieren:
# Alle schließen (Kamera-App, Teams/Zoom, Browser, Windows Hello), dann in PowerShell neu anhängen.

# ffplay funktioniert, OpenCV nicht → V4L2 + MJPEG erzwingen, ggf. Auflösung/FPS senken.
```
